In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn import metrics

data_ch = pd.read_csv('../data/raw/601988.SH.csv')
data_google = pd.read_csv('../data/raw/Google_Stock_Price_Train.csv')

data_ch.index = pd.to_datetime(data_ch['trade_date'], format='%Y%m%d') 
data_google.index = pd.to_datetime(data_google['Date'],format='%m/%d/%Y')


In [18]:
data_ch = data_ch.drop(['ts_code','trade_date'], axis=1)
data_ch.head()

,open,high,low,close,pre_close,change,pct_chg,vol,amount,turnover_rate,volume_ratio,pe,pb,ps,total_share,float_share,free_share,total_mv,circ_mv
trade_date,,,,,,,,,,,,,,,,,,,
2007-01-04,5.69,5.97,5.37,5.63,5.43,0.20,3.68,7281333.33,4224852.614,16.8115,1.67,34.1143,3.7772,9.8280,2.538392e+07,433117.0531,345454.7,1.429114e+08,2438449.009
2007-01-05,5.30,5.34,5.07,5.07,5.63,-0.56,-9.95,7867306.84,4050377.943,15.1068,1.60,30.7210,3.4015,8.8504,2.538392e+07,520779.4000,345454.7,1.286965e+08,2640351.558
2007-01-08,4.87,5.14,4.83,5.08,5.07,0.01,0.20,5565756.49,2763751.983,10.6874,0.97,30.7816,3.4082,8.8679,2.538392e+07,520779.4000,345454.7,1.289503e+08,2645559.352
2007-01-09,5.06,5.19,4.95,5.18,5.08,0.10,1.97,4334458.33,2198327.249,8.3230,0.68,31.3875,3.4753,9.0424,2.538392e+07,520779.4000,345454.7,1.314887e+08,2697637.292
2007-01-10,5.25,5.29,5.05,5.10,5.18,-0.08,-1.54,3721171.25,1915221.182,7.1454,0.59,30.9028,3.4217,8.9028,2.538392e+07,520779.4000,345454.7,1.294580e+08,2655974.940


In [15]:
data_google = data_google.drop(['Date'], axis=1)
data_google.head()

,Open,High,Low,Close,Volume
Date,,,,,
2012-01-03,325.25,332.83,324.97,663.59,"7,380,500"
2012-01-04,331.27,333.87,329.08,666.45,"5,749,400"
2012-01-05,329.83,330.75,326.89,657.21,"6,590,300"
2012-01-06,328.34,328.77,323.68,648.24,"5,405,900"
2012-01-09,322.04,322.29,309.46,620.76,"11,688,800"
